# SCENIC+ and PINNACLE Integration: Interactive Analysis

This notebook demonstrates the interactive analysis capabilities of the SCENIC+ and PINNACLE integration framework.

## Overview

This tutorial covers:
1. Loading and preprocessing data
2. Network integration
3. Differential analysis
4. Interactive visualizations
5. Results interpretation

**Author:** Manus AI  
**Date:** July 2025

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import the integration package
from scenic_pinnacle import (
    ScenicPinnacleIntegrator,
    ScenicProcessor,
    PinnacleProcessor,
    NetworkIntegrator,
    DifferentialAnalyzer,
    NetworkVisualizer
)

# Set up plotting
%matplotlib inline
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

## 1. Configuration and Setup

First, let's set up the configuration for our analysis.

In [ ]:
# Configuration for the integration
config = {
    'scenic': {
        'min_regulon_size': 10,
        'min_target_genes': 5,
        'importance_threshold': 0.1
    },
    'pinnacle': {
        'embedding_dim': 256,
        'normalize_embeddings': True,
        'context_threshold': 0.1
    },
    'integration': {
        'similarity_threshold': 0.5,
        'min_overlap': 3
    },
    'differential': {
        'pvalue_threshold': 0.05,
        'fold_change_threshold': 1.5
    },
    'visualization': {
        'layout_algorithm': 'spring',
        'node_size_range': (20, 200),
        'edge_width_range': (0.5, 3.0)
    }
}

# Data paths (modify these to point to your data)
data_dir = Path("../data")
scenic_data_path = data_dir / "scenic_networks.pkl"
pinnacle_data_path = data_dir / "pinnacle_embeddings.pkl"
output_dir = Path("../results/interactive_analysis")

# Create output directory
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Configuration set up successfully!")
print(f"Output directory: {output_dir}")

## 2. Initialize the Integrator

Create the main integrator object that will coordinate the analysis.

In [ ]:
# Initialize the integrator
integrator = ScenicPinnacleIntegrator(config=config)

print("Integrator initialized successfully!")
print(f"Available processors:")
print(f"  - SCENIC+ processor: {type(integrator.scenic_processor).__name__}")
print(f"  - PINNACLE processor: {type(integrator.pinnacle_processor).__name__}")
print(f"  - Network integrator: {type(integrator.network_integrator).__name__}")
print(f"  - Differential analyzer: {type(integrator.differential_analyzer).__name__}")
print(f"  - Visualizer: {type(integrator.visualizer).__name__}")

## 3. Generate Example Data

For this tutorial, we'll generate example data. In practice, you would load your own SCENIC+ and PINNACLE results.

In [ ]:
def generate_example_data():
    """Generate example SCENIC+ and PINNACLE data."""
    import pickle
    
    # Create data directory
    data_dir.mkdir(parents=True, exist_ok=True)
    
    # Generate SCENIC+ networks
    scenic_networks = {}
    conditions = ['healthy', 'disease', 'treatment']
    
    for i, condition in enumerate(conditions):
        G = nx.DiGraph()
        
        # Add transcription factors
        tfs = [f'TF_{j}' for j in range(1, 16)]  # 15 TFs
        for tf in tfs:
            G.add_node(tf, node_type='TF')
        
        # Add target genes
        targets = [f'Gene_{j}' for j in range(1, 101)]  # 100 targets
        for target in targets:
            G.add_node(target, node_type='target')
        
        # Add regulatory edges with condition-specific patterns
        np.random.seed(42 + i)
        for tf_idx, tf in enumerate(tfs):
            # Each TF regulates 10-20 targets
            n_targets = np.random.randint(10, 21)
            
            # Add condition-specific bias
            if condition == 'disease':
                # Disease condition has stronger regulation for some TFs
                if tf_idx < 5:
                    importance_boost = 0.3
                else:
                    importance_boost = 0.0
            elif condition == 'treatment':
                # Treatment condition has different pattern
                if tf_idx >= 10:
                    importance_boost = 0.4
                else:
                    importance_boost = -0.1
            else:
                importance_boost = 0.0
            
            selected_targets = np.random.choice(targets, n_targets, replace=False)
            
            for target in selected_targets:
                base_importance = np.random.uniform(0.2, 0.8)
                importance = np.clip(base_importance + importance_boost, 0.1, 1.0)
                
                G.add_edge(tf, target, 
                          importance=importance,
                          edge_type='regulation',
                          enhancer=f'Enhancer_{np.random.randint(1, 1000)}')
        
        scenic_networks[condition] = G
    
    # Save SCENIC+ data
    with open(scenic_data_path, 'wb') as f:
        pickle.dump(scenic_networks, f)
    
    # Generate PINNACLE embeddings
    pinnacle_embeddings = {}
    
    # Create protein list
    all_genes = [f'TF_{j}' for j in range(1, 16)] + [f'Gene_{j}' for j in range(1, 101)]
    protein_ids = [f'Protein_{gene}' for gene in all_genes]
    
    for i, condition in enumerate(conditions):
        np.random.seed(123 + i)
        
        # Generate embeddings with condition-specific structure
        embeddings = np.random.randn(len(protein_ids), 256) * 0.5
        
        # Add condition-specific clustering
        if condition == 'healthy':
            # TF proteins form one cluster
            embeddings[:15] += np.array([1.0, 0.0] + [0.0] * 254)
            # Target proteins form another cluster
            embeddings[15:] += np.array([0.0, 1.0] + [0.0] * 254)
        elif condition == 'disease':
            # Different clustering pattern
            embeddings[:15] += np.array([0.5, 0.5] + [0.0] * 254)
            embeddings[15:50] += np.array([1.5, 0.0] + [0.0] * 254)
            embeddings[50:] += np.array([0.0, 1.5] + [0.0] * 254)
        else:  # treatment
            # Treatment restores some structure
            embeddings[:15] += np.array([0.8, 0.2] + [0.0] * 254)
            embeddings[15:] += np.array([0.2, 0.8] + [0.0] * 254)
        
        # Normalize embeddings
        norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
        norms[norms == 0] = 1
        embeddings = embeddings / norms
        
        pinnacle_embeddings[condition] = {
            'embeddings': embeddings,
            'protein_ids': protein_ids,
            'embedding_dim': 256
        }
    
    # Save PINNACLE data
    with open(pinnacle_data_path, 'wb') as f:
        pickle.dump(pinnacle_embeddings, f)
    
    return scenic_networks, pinnacle_embeddings

# Generate example data
print("Generating example data...")
scenic_networks, pinnacle_embeddings = generate_example_data()

print(f"Generated SCENIC+ networks for conditions: {list(scenic_networks.keys())}")
print(f"Generated PINNACLE embeddings for conditions: {list(pinnacle_embeddings.keys())}")

# Display network statistics
print("\nNetwork Statistics:")
for condition, network in scenic_networks.items():
    print(f"  {condition}: {network.number_of_nodes()} nodes, {network.number_of_edges()} edges")

## 4. Load Data

Load the SCENIC+ and PINNACLE data into the integrator.

In [ ]:
# Load SCENIC+ data
print("Loading SCENIC+ data...")
integrator.load_scenic_data(scenic_data_path, data_format='pickle')
print(f"Loaded {len(integrator.scenic_networks)} SCENIC+ networks")

# Load PINNACLE data
print("\nLoading PINNACLE data...")
integrator.load_pinnacle_data(pinnacle_data_path, data_format='pickle')
print(f"Loaded embeddings for {len(integrator.pinnacle_embeddings)} contexts")

# Display loaded data summary
print("\nLoaded Data Summary:")
print("SCENIC+ Networks:")
for condition, network in integrator.scenic_networks.items():
    tfs = sum(1 for n in network.nodes() if network.nodes[n].get('node_type') == 'TF')
    targets = sum(1 for n in network.nodes() if network.nodes[n].get('node_type') == 'target')
    print(f"  {condition}: {tfs} TFs, {targets} targets, {network.number_of_edges()} regulations")

print("\nPINNACLE Embeddings:")
for condition, embeddings in integrator.pinnacle_embeddings.items():
    print(f"  {condition}: {len(embeddings['protein_ids'])} proteins, {embeddings['embedding_dim']}D embeddings")

## 5. Network Integration

Integrate the SCENIC+ regulatory networks with PINNACLE protein embeddings.

In [ ]:
# Perform network integration
print("Integrating networks...")
integrated_networks = integrator.integrate_networks()

print(f"\nIntegration completed! Created {len(integrated_networks)} integrated networks.")

# Display integration statistics
print("\nIntegrated Network Statistics:")
for condition, network in integrated_networks.items():
    print(f"\n{condition}:")
    print(f"  Nodes: {network.number_of_nodes()}")
    print(f"  Edges: {network.number_of_edges()}")
    print(f"  Density: {nx.density(network):.4f}")
    
    # Edge type distribution
    edge_types = {}
    for _, _, data in network.edges(data=True):
        edge_type = data.get('edge_type', 'unknown')
        edge_types[edge_type] = edge_types.get(edge_type, 0) + 1
    
    print(f"  Edge types: {edge_types}")
    
    # Node type distribution
    node_types = {}
    for node in network.nodes():
        node_type = network.nodes[node].get('node_type', 'unknown')
        node_types[node_type] = node_types.get(node_type, 0) + 1
    
    print(f"  Node types: {node_types}")

## 6. Differential Analysis

Perform differential analysis between conditions to identify changes in regulatory and protein networks.

In [ ]:
# Perform differential analysis between healthy and disease conditions
print("Performing differential analysis: healthy vs disease...")
diff_results_1 = integrator.differential_analysis('healthy', 'disease', analysis_type='both')

# Perform differential analysis between disease and treatment conditions
print("Performing differential analysis: disease vs treatment...")
diff_results_2 = integrator.differential_analysis('disease', 'treatment', analysis_type='both')

# Display differential analysis results
print("\nDifferential Analysis Results:")
print("\n1. Healthy vs Disease:")
summary_1 = diff_results_1.get('summary', {})
for analysis_type, stats in summary_1.items():
    print(f"   {analysis_type.title()}:")
    for stat_name, value in stats.items():
        print(f"     {stat_name}: {value}")

print("\n2. Disease vs Treatment:")
summary_2 = diff_results_2.get('summary', {})
for analysis_type, stats in summary_2.items():
    print(f"   {analysis_type.title()}:")
    for stat_name, value in stats.items():
        print(f"     {stat_name}: {value}")

## 7. Visualizations

Create various visualizations to explore the integrated networks and differential analysis results.

In [ ]:
# Create network visualizations
print("Creating network visualizations...")

# Visualize each integrated network
for condition in ['healthy', 'disease', 'treatment']:
    if condition in integrated_networks:
        print(f"\nVisualizing {condition} network...")
        
        # Create output directory for this condition
        condition_dir = output_dir / condition
        condition_dir.mkdir(exist_ok=True)
        
        # Generate different types of plots
        integrator.visualize_networks(
            condition, 
            condition_dir,
            plot_types=['network', 'heatmap']
        )
        
        print(f"   Saved visualizations to {condition_dir}")

print("\nNetwork visualizations completed!")

In [ ]:
# Create differential analysis visualizations
print("Creating differential analysis visualizations...")

# Visualize differential results
visualizer = integrator.visualizer

# Healthy vs Disease
diff_dir_1 = output_dir / 'differential_healthy_vs_disease'
diff_dir_1.mkdir(exist_ok=True)

visualizer.plot_differential_analysis(
    diff_results_1,
    diff_dir_1 / 'volcano_plot.png',
    plot_type='volcano'
)

visualizer.plot_differential_analysis(
    diff_results_1,
    diff_dir_1 / 'summary_heatmap.png',
    plot_type='heatmap'
)

visualizer.plot_differential_analysis(
    diff_results_1,
    diff_dir_1 / 'summary_barplot.png',
    plot_type='barplot'
)

# Disease vs Treatment
diff_dir_2 = output_dir / 'differential_disease_vs_treatment'
diff_dir_2.mkdir(exist_ok=True)

visualizer.plot_differential_analysis(
    diff_results_2,
    diff_dir_2 / 'volcano_plot.png',
    plot_type='volcano'
)

visualizer.plot_differential_analysis(
    diff_results_2,
    diff_dir_2 / 'summary_heatmap.png',
    plot_type='heatmap'
)

visualizer.plot_differential_analysis(
    diff_results_2,
    diff_dir_2 / 'summary_barplot.png',
    plot_type='barplot'
)

print("Differential analysis visualizations completed!")

## 8. Export Results

Export all results in multiple formats for further analysis.

In [ ]:
# Export all results
print("Exporting results...")

results_dir = output_dir / 'exported_results'
integrator.export_results(results_dir, formats=['pickle', 'csv', 'json'])

print(f"Results exported to {results_dir}")

# List exported files
print("\nExported files:")
for file_path in sorted(results_dir.glob('*')):
    print(f"  {file_path.name}")

## 9. Quality Assessment

Assess the quality of the integration and analysis results.

In [ ]:
# Compute quality metrics
print("Computing quality metrics...")

quality_metrics = integrator._compute_quality_metrics()

print("\nQuality Assessment:")
for metric_type, metrics in quality_metrics.items():
    print(f"\n{metric_type.replace('_', ' ').title()}:")
    
    if isinstance(metrics, dict):
        for condition, condition_metrics in metrics.items():
            print(f"  {condition}:")
            if isinstance(condition_metrics, dict):
                for metric_name, value in condition_metrics.items():
                    if isinstance(value, float):
                        print(f"    {metric_name}: {value:.4f}")
                    else:
                        print(f"    {metric_name}: {value}")
            else:
                print(f"    {condition_metrics}")
    else:
        print(f"  {metrics}")

## 10. Summary and Interpretation

Summarize the key findings from the integration analysis.

In [ ]:
# Generate analysis summary
print("Analysis Summary")
print("=" * 50)

print(f"\n1. Data Integration:")
print(f"   - Integrated {len(integrated_networks)} conditions")
print(f"   - Conditions analyzed: {', '.join(integrated_networks.keys())}")

print(f"\n2. Network Statistics:")
for condition, network in integrated_networks.items():
    density = nx.density(network)
    clustering = nx.average_clustering(network) if network.number_of_nodes() > 0 else 0
    print(f"   {condition}: {network.number_of_nodes()} nodes, {network.number_of_edges()} edges")
    print(f"     Density: {density:.4f}, Clustering: {clustering:.4f}")

print(f"\n3. Differential Analysis:")
print(f"   - Performed {len(integrator.differential_results)} comparisons")
for comparison, results in integrator.differential_results.items():
    print(f"   {comparison}:")
    summary = results.get('summary', {})
    for analysis_type, stats in summary.items():
        significant = stats.get('significant_changes', 0)
        print(f"     {analysis_type}: {significant} significant changes")

print(f"\n4. Output Files:")
print(f"   - Results exported to: {results_dir}")
print(f"   - Visualizations saved to: {output_dir}")

print(f"\n5. Key Findings:")
print(f"   - Successfully integrated regulatory and protein networks")
print(f"   - Identified condition-specific network changes")
print(f"   - Generated comprehensive visualizations")
print(f"   - Provided quantitative differential analysis")

print("\n" + "=" * 50)
print("Analysis completed successfully!")
print("Check the output directory for detailed results and visualizations.")

## Next Steps

This notebook demonstrated the basic workflow for SCENIC+ and PINNACLE integration. Here are some suggestions for further analysis:

1. **Biological Interpretation**: 
   - Analyze the biological significance of identified regulatory changes
   - Perform pathway enrichment analysis on differential genes/proteins
   - Validate key findings with experimental data

2. **Advanced Analysis**:
   - Explore network modules and communities
   - Perform time-series analysis if temporal data is available
   - Integrate additional omics layers (metabolomics, epigenomics)

3. **Visualization Enhancement**:
   - Create interactive network visualizations
   - Generate publication-ready figures
   - Develop custom visualization for specific biological questions

4. **Method Development**:
   - Optimize integration parameters for your specific dataset
   - Implement custom quality control metrics
   - Develop specialized analysis workflows

For more information and advanced tutorials, please refer to the documentation and additional examples in the repository.